In [1]:
from VariationalBayes import ScalarParam, ModelParamsDict, VectorParam, PosDefMatrixParam
from VariationalBayes.NormalParams import MVNParam, UVNParam, UVNParamVector
from VariationalBayes.GammaParams import GammaParam
from VariationalBayes.ExponentialFamilies import \
    UnivariateNormalEntropy, MultivariateNormalEntropy, GammaEntropy, \
    MVNPrior, UVNPrior, GammaPrior


from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np
import autograd.numpy.random as npr
import autograd.scipy as asp
import scipy as sp

import copy
from scipy import optimize

In [2]:
# Load data saved by stan_results_to_json.R and run_stan.R in LRVBLogitGLMM.
import os
import json

simulate_data = False
prior_par = ModelParamsDict('Prior Parameters')

if not simulate_data:
    analysis_name = 'simulated_data_small'
    data_dir = os.path.join(os.environ['GIT_REPO_LOC'], 'LRVBLogitGLMM/LogitGLMMLRVB/inst/data/')
    json_filename = os.path.join(data_dir, '%s_stan_dat.json' % analysis_name)
    json_output_filename = os.path.join(data_dir, '%s_python_vb_results.json' % analysis_name)

    json_file = open(json_filename, 'r')
    stan_dat = json.load(json_file)
    json_file.close()

    print stan_dat.keys()
    K = stan_dat['K'][0]
    NObs = stan_dat['N'][0]
    NG = stan_dat['NG'][0]
    N = NObs / NG
    y_g_vec = np.array(stan_dat['y_group'])
    y_vec = np.array(stan_dat['y'])
    x_mat = np.array(stan_dat['x'])
    
    # Define a class to contain prior parameters.
    prior_par.push_param(VectorParam('beta_prior_mean', K, val=np.array(stan_dat['beta_prior_mean'])))
    prior_par.push_param(PosDefMatrixParam('beta_prior_var', K, val=np.array(stan_dat['beta_prior_var'])))

    prior_par.push_param(ScalarParam('mu_prior_mean', val=stan_dat['mu_prior_mean'][0]))
    prior_par.push_param(ScalarParam('mu_prior_var', val=stan_dat['mu_prior_var'][0]))

    prior_par.push_param(ScalarParam('tau_prior_alpha', val=stan_dat['tau_prior_alpha'][0]))
    prior_par.push_param(ScalarParam('tau_prior_beta', val=stan_dat['tau_prior_beta'][0]))

    # An index set to make sure jacobians match the order expected by R.
    prior_par_indices = copy.deepcopy(prior_par)
    prior_par_indices.set_name('Prior Indices')
    prior_par_indices.set_vector(np.array(range(prior_par_indices.vector_size())))
else:
    # Simulate data instead of loading it if you like
    N = 200     # observations per group
    K = 5      # dimension of regressors
    NG = 200      # number of groups

    # Generate data
    def Logistic(u):
        return np.exp(u) / (1 + np.exp(u))

    NObs = NG * N
    true_beta = np.array(range(5))
    true_beta = true_beta - np.mean(true_beta)
    true_mu = 0.
    true_tau = 40.0
    true_u = np.random.normal(true_mu, 1 / np.sqrt(true_tau), NG)

    x_mat = np.random.random(K * NObs).reshape(NObs, K) - 0.5
    y_g_vec = [ g for g in range(NG) for n in range(N) ]
    true_rho = Logistic(np.matmul(x_mat, true_beta) + true_u[y_g_vec])
    y_vec = np.random.random(NObs) < true_rho
    
    prior_par.push_param(VectorParam('beta_prior_mean', K, val=np.zeros(K)))
    prior_par.push_param(PosDefMatrixParam('beta_prior_var', K, val=10 * np.eye(K)))

    prior_par.push_param(ScalarParam('mu_prior_mean', val=0))
    prior_par.push_param(ScalarParam('mu_prior_var', val=2))

    prior_par.push_param(ScalarParam('tau_prior_alpha', val=3.0))
    prior_par.push_param(ScalarParam('tau_prior_beta', val=10.0))

print N * NG
print np.mean(y_vec)

[u'y_group', u'mu_prior_var', u'mu_prior_t', u'mu_prior_var_c', u'K', u'beta_prior_var', u'tau_prior_beta', u'N', u'mu_prior_mean_c', u'mu_prior_epsilon', u'mu_prior_mean', u'y', u'x', u'NG', u'beta_prior_mean', u'tau_prior_alpha']
1000
0.324


In [3]:
# Build an object to contain a variational approximation to a K-dimensional multivariate normal.
glmm_par = ModelParamsDict('GLMM Parameters')

glmm_par.push_param(UVNParam('mu'))
glmm_par.push_param(GammaParam('tau'))
glmm_par.push_param(MVNParam('beta', K))
glmm_par.push_param(UVNParamVector('u', NG))

glmm_par['mu'].mean.set(0.1)
glmm_par['mu'].var.set(1.0)

glmm_par['tau'].shape.set(2.1)
glmm_par['tau'].rate.set(2.1)

glmm_par['beta'].mean.set(np.full(K, 0.))
glmm_par['beta'].cov.set(20 * np.eye(K))

glmm_par['u'].mean.set(np.full(NG, 0.))
glmm_par['u'].var.set(np.full(NG, 1))


In [4]:
moment_par = ModelParamsDict('Moment Parameters')
moment_par.push_param(VectorParam('e_beta', K))
moment_par.push_param(ScalarParam('e_mu'))
moment_par.push_param(ScalarParam('e_tau'))
moment_par.push_param(ScalarParam('e_log_tau'))
moment_par.push_param(VectorParam('e_u', NG))

def set_moments(glmm_par, moment_par):
    moment_par['e_beta'].set(glmm_par['beta'].e())
    moment_par['e_mu'].set(glmm_par['mu'].e())
    moment_par['e_tau'].set(glmm_par['tau'].e())
    moment_par['e_log_tau'].set(glmm_par['tau'].e_log())
    moment_par['e_u'].set(glmm_par['u'].e())
    
set_moments(glmm_par, moment_par)


In [5]:
def ELogPrior(prior_par, glmm_par_elbo):
    e_beta = glmm_par_elbo['beta'].mean.get()
    cov_beta = glmm_par_elbo['beta'].cov.get()
    beta_prior_info = np.linalg.inv(prior_par['beta_prior_var'].get())
    beta_prior_mean = prior_par['beta_prior_mean'].get()
    e_log_p_beta = MVNPrior(beta_prior_mean, beta_prior_info, e_beta, cov_beta)
    
    e_mu = glmm_par_elbo['mu'].mean.get()
    var_mu = glmm_par_elbo['mu'].var.get()
    e_log_p_mu = UVNPrior(prior_par['mu_prior_mean'].get(), 1 / prior_par['mu_prior_var'].get(), e_mu, var_mu) 

    e_tau = glmm_par_elbo['tau'].e()
    e_log_tau = glmm_par_elbo['tau'].e_log()
    tau_prior_shape = prior_par['tau_prior_alpha'].get()
    tau_prior_rate = prior_par['tau_prior_beta'].get()
    e_log_p_tau = GammaPrior(tau_prior_shape, tau_prior_rate, e_tau, e_log_tau)
    
    return  e_log_p_beta + e_log_p_mu + e_log_p_tau
           

def DataLogLikelihood(x_mat, y_vec, e_beta, cov_beta, e_u, var_u, std_draws):
    z_mean = e_u + np.matmul(x_mat, e_beta)
    z_sd = np.sqrt(var_u + np.einsum('nk,kj,nj->n', x_mat, cov_beta, x_mat))
    z = np.einsum('i,j->ij', z_sd, std_draws) + np.expand_dims(z_mean, 1)

    # The sum is over observations and draws, so dividing by the draws size
    # gives the sum of sample expectations over the draws.
    # p = exp(z) / (1 + exp(z))
    # log(1 - p) = log(1 / (1 + exp(z))) = -log(1 + exp(z))
    logit_term = -np.sum(np.log1p(np.exp(z))) / std_draws.size
    y_term = np.sum(y_vec * z_mean)
    return y_term + logit_term


def RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau):
    return -0.5 * e_tau * np.sum(((e_mu - e_u) ** 2) + var_mu + var_u) + 0.5 * e_log_tau * len(e_u)

    
def Elbo(y_vec, x_mat, y_g_vec, glmm_par_elbo, std_draws, prior_par):
    e_beta = glmm_par_elbo['beta'].mean.get()
    cov_beta = glmm_par_elbo['beta'].cov.get()
    
    e_u = glmm_par_elbo['u'].mean.get()
    var_u = glmm_par_elbo['u'].var.get()
    
    e_mu = glmm_par_elbo['mu'].mean.get()
    var_mu = glmm_par_elbo['mu'].var.get()
    
    e_tau = glmm_par_elbo['tau'].e()
    e_log_tau = glmm_par_elbo['tau'].e_log()
    
    ll = \
        DataLogLikelihood(x_mat, y_vec, e_beta, cov_beta,
                          e_u[y_g_vec], var_u[y_g_vec], std_draws) + \
        RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau)

    e_log_prior = ELogPrior(prior_par, glmm_par_elbo)
    
    tau_shape = glmm_par_elbo['tau'].shape.get()
    tau_rate = glmm_par_elbo['tau'].rate.get()
    entropy = \
        UnivariateNormalEntropy(var_mu) + \
        MultivariateNormalEntropy(cov_beta) + \
        UnivariateNormalEntropy(var_u) + \
        GammaEntropy(tau_shape, tau_rate)

    return ll[0] + e_log_prior[0] + entropy


class KLWrapper():
    def __init__(self, glmm_par, moment_par, prior_par, x_mat, y_vec, y_g_vec, num_draws):
        self.__glmm_par_ad = copy.deepcopy(glmm_par)
        self.__moment_par = copy.deepcopy(moment_par)
        self.__prior_par_ad = copy.deepcopy(prior_par)
        self.x_mat = x_mat
        self.y_vec = y_vec
        self.y_g_vec = y_g_vec
        draw_spacing = 1 / float(num_draws + 1)
        target_quantiles = np.linspace(draw_spacing, 1 - draw_spacing, num_draws)
        self.std_draws = sp.stats.norm.ppf(target_quantiles)

    def Eval(self, free_par_vec, verbose=False):
        self.__glmm_par_ad.set_free(free_par_vec)
        kl = -Elbo(self.y_vec, self.x_mat, self.y_g_vec,
                   self.__glmm_par_ad, self.std_draws, self.__prior_par_ad)
        if verbose: print kl
            
        # TODO: this is returning an array when it should be a scalar.
        return kl

    # Return a posterior moment of interest as a function of
    # unconstrained parameters.  In this case it is a bit silly,
    # but in full generality posterior moments may be a complicated
    # function of moment parameters.
    def GetMoments(self, free_par_vec):
        self.__glmm_par_ad.set_free(free_par_vec)
        set_moments(self.__glmm_par_ad, self.__moment_par)
        return self.__moment_par.get_vector()


# TODO: get the log prior derivatives, too.
    
kl_wrapper = KLWrapper(glmm_par, moment_par, prior_par, x_mat, y_vec, y_g_vec, 10)
KLGrad = grad(kl_wrapper.Eval)
KLHess = hessian(kl_wrapper.Eval)
MomentJacobian = jacobian(kl_wrapper.GetMoments)
KLHessVecProd = hessian_vector_product(kl_wrapper.Eval)  
free_par_vec = glmm_par.get_free()
kl_wrapper.Eval(free_par_vec)

2510.5762058064029

In [7]:
import timeit

time_num = 10

print 'Function time:'
print timeit.timeit(lambda: kl_wrapper.Eval(free_par_vec), number=time_num) / time_num

print 'Grad time:'
print timeit.timeit(lambda: KLGrad(free_par_vec), number=time_num) / time_num

print 'Hessian vector product time:'
print timeit.timeit(lambda: KLHessVecProd(free_par_vec, free_par_vec + 1), number=time_num) / time_num

print 'Moment jacobian time:'
print timeit.timeit(lambda: MomentJacobian(free_par_vec), number=time_num) / time_num

# so slow
# print 'Hessian time:'
# print timeit.timeit(lambda: KLHess(free_par_vec), number=time_num) / time_num


Function time:
0.00187618732452
Grad time:
0.00822260379791
Hessian vector product time:
0.0107099056244
Moment jacobian time:
0.0593795061111


In [12]:
import time

init_par_vec = free_par_vec

# Optimize.
vb_time = time.time()
print 'Running BFGS'
vb_opt_bfgs = optimize.minimize(
    lambda par: kl_wrapper.Eval(par, verbose=True), init_par_vec,
    method='bfgs', jac=KLGrad, tol=1e-2, options={'maxiter': 5000, 'disp': True})

init_par_vec = free_par_vec
print 'Running Newton Trust Region'
vb_opt = optimize.minimize(
    lambda par: kl_wrapper.Eval(par, verbose=True),
    vb_opt_bfgs.x, method='trust-ncg', jac=KLGrad, hessp=KLHessVecProd, options={'maxiter': 5000, 'disp': True})

vb_time = time.time() - vb_time

glmm_par_opt = copy.deepcopy(glmm_par)
glmm_par_opt.set_free(vb_opt.x)
moment_jac = MomentJacobian(vb_opt.x)

print 'Done.'

print vb_time / 60

Running BFGS
2510.57620581
2040.57637476
1348.98986883
811.559883895
1548.61181724
681.981760308
753.917446396
630.165475278
567.962615493
463.179050012
316.612561758
505.190228987
280.202894241
294.138215785
275.497415873
266.4350659
238.304431917
197.540979229
213.04439515
182.154675694
169.624964606
150.219791665
133.342883533
156.982350356
130.671090635
126.83802
119.777898975
111.381531185
103.558520907
91.3498624967
92.3153882053
88.3959992307
87.0860869676
87.8100870583
86.1667421012
84.4117429925
81.5248204288
76.9098666011
71.5520110045
66.7795338224
71.7546599181
66.1876936725
65.3117862818
65.3466244243
64.8878686614
64.1955175632
63.0874418169
61.5172448741
60.2222870158
59.5265951402
59.1000504279
58.9366730512
58.8384712485
58.7611087707
58.6982178059
58.6249126185
58.5282973956
58.3990821163
58.2668659469
58.1504705741
58.0628175495
58.0156905879
57.9825284173
57.9685841441
57.959142624
57.9546398084
57.9521649933
57.9509632242
57.9504135059
57.9501692733
57.949998558
57

In [9]:
# print(glmm_par_opt)
if simulate_data:
    print true_beta
    print glmm_par_opt['beta']
    print '---------------\n'
    print true_tau
    print glmm_par_opt['tau'].e()

    e_u = glmm_par_opt['u'].e()
    var_u = glmm_par_opt['u'].var.get()
    e_beta = glmm_par_opt['beta'].e()
    e_beta_outer = glmm_par_opt['beta'].e_outer()
    std_draws = kl_wrapper.std_draws

    rho_mean = e_u[y_g_vec] + np.matmul(x_mat, e_beta)
    rho_sd = np.sqrt(var_u[y_g_vec] + np.einsum('nk,kj,nj->n', x_mat, e_beta_outer, x_mat))
    z = np.einsum('i,j->ij', rho_sd, std_draws) + np.expand_dims(rho_mean, 1)
    logit_term = -np.einsum('ij->i', np.log1p(np.exp(z))) / std_draws.size

    print rho_sd
    print var_u[y_g_vec]
    # print np.mean(var_u)


In [10]:
# Check the random effect estimates.  This requires simulated data.
if simulate_data:
    from ggplot import *
    import pandas as pd
    %matplotlib inline
    
    print glmm_par_opt['mu'].e()
    print true_mu

    print glmm_par_opt['tau'].e()
    print true_tau

    plot_df = pd.DataFrame({ 'opt': glmm_par_opt['u'].mean.get(), 'true': true_u })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)
    
    plot_df = pd.DataFrame({ 'opt': glmm_par_opt['beta'].mean.get(), 'true': true_beta })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)
    
    plot_df = pd.DataFrame({ 'opt': logit_term, 'true': np.log(1 - true_rho) })
    print ggplot(plot_df, aes(x='true', y='opt')) + geom_point() + geom_abline(slope=1, intercept=0)

In [26]:
# LRVB with conjugate gradient.  This turns out to be way slower with any appreciable number of moments.
if False:
    from scipy.sparse.linalg import LinearOperator
    import sys

    # This will actually compute Hess^1 * moment_jac.T, leading to perhaps confusing
    # naming of "columns".  
    ObjHessVecProdLO = LinearOperator((vb_opt.x.size, vb_opt.x.size), lambda par: KLHessVecProd(vb_opt.x, par))
    # print moment_jac.T.shape
    # print ObjHessVecProdLO.shape
    # cg_res, info = scipy.sparse.linalg.cg(ObjHessVecProdLO, moment_jac.T)

    cg_time = time.time()
    lrvb_term = np.full(moment_jac.T.shape, float('nan'))
    for col in range(moment_jac.shape[0]):
        sys.stdout.write('.')
        sys.stdout.flush()
        cg_res, info = sp.sparse.linalg.cg(ObjHessVecProdLO, moment_jac[col, :])
        assert info == 0
        lrvb_term[:, col] = cg_res
    cg_time = time.time() - cg_time

    print 'all done dude'
else:
    cg_time = float('inf')

In [27]:
# Slow, but probably faster than using CG.
hess_time = time.time()
kl_hess = KLHess(vb_opt.x)
hess_time =  time.time() - hess_time
elbo_hess = -kl_hess

print 'hess_time: %f' % hess_time
print 'cg_time: %f' % cg_time

hess_time: 1.212434
cg_time: inf


In [33]:
lrvb_cov = np.matmul(moment_jac, np.linalg.solve(kl_hess[:, :], moment_jac.T))

moment_indices = copy.deepcopy(moment_par)
moment_indices.set_vector(1 + np.array(range(moment_indices.vector_size())))
print moment_indices.dictval()

{'e_beta': [1, 2, 3, 4, 5], 'e_log_tau': [8], 'e_u': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108], 'e_mu': [6], 'e_tau': [7]}


In [34]:
if not simulate_data:
    result_dict = { 'glmm_par_opt': glmm_par_opt.dictval(), 'vb_time': vb_time,
                    'moment_jac': moment_jac.tolist(), 'moment_indices': moment_indices.dictval(),
                    'hess_time': elbo_hess.tolist(), 'lrvb_cov': lrvb_cov.tolist(), 'elbo_hess': elbo_hess.tolist() }

    result_json = json.dumps(result_dict)
    json_file = open(json_output_filename, 'w')
    json_file.write(result_json)
    json_file.close()

    print(json_output_filename)



/home/rgiordan/Documents/git_repos/LRVBLogitGLMM/LogitGLMMLRVB/inst/data/simulated_data_small_python_vb_results.json
